In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from matplotlib.pyplot import hist, show
from collections import Counter
import urllib
import codecs
from nltk.stem import WordNetLemmatizer

## Czytania danego dnia

In [2]:
#downloading the webpage
import requests
import datetime

now = datetime.datetime.now()
year = str(now.year)
month = str(now.month)
if len(month) == 1:
    month = "0" + month
day = str(now.day)
ymd = year+month+day

    
address = "http://mateusz.pl/czytania/" + year + "/" + ymd + ".html"
#address = "http://mateusz.pl/czytania/2018/20180909.html" #niedziela
page = requests.get(address)

In [3]:
#parsing a page
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
#print(soup.prettify()) 
#Exception handling for more readings
try:
    section = list(soup.children)[9]
    section_content = section.find_all('p')
except:
    section = soup.find_all('section')[0]
    section_content =section.find_all('p')

In [5]:
len(section_content)

7

In [6]:
pierwsze_czytanie = section_content[0]
pierwsze_czytanie

<p>(1 Kor 7, 25-31)<br/>
    Bracia: Nie mam nakazu Pańskiego co do dziewic, lecz daję radę jako ten, który – wskutek doznanego od Pana miłosierdzia – godzien jest, aby mu wierzono. Uważam, iż przy obecnych utrapieniach dobrze jest tak zostać, dobrze to dla człowieka tak żyć. Jesteś związany z żoną? Nie usiłuj odłączać się od niej! Jesteś wolny? Nie szukaj żony! Ale jeśli się ożenisz, nie grzeszysz. Podobnie i dziewica, jeśli wychodzi za mąż, nie grzeszy. Tacy jednak cierpieć będą udręki ciała, a ja chciałbym ich wam oszczędzić. Mówię wam, bracia, czas jest krótki. Trzeba więc, aby ci, którzy mają żony, tak żyli, jakby byli nieżonaci, a ci, którzy płaczą, tak jakby nie płakali, ci zaś, którzy się radują, tak jakby się nie radowali; ci zaś, którzy nabywają, jak gdyby nie posiadali; ci, którzy używają tego świata, tak jakby z niego nie korzystali. Przemija bowiem postać tego świata.</p>

In [12]:
ewangelia = section_content[len(section_content)-1]
ewangelia

<p>(Łk 6, 20-26)<br/>
		Jezus podniósł oczy na swoich uczniów i mówił: "Błogosławieni jesteście, ubodzy, albowiem do was należy królestwo Boże. Błogosławieni, którzy teraz głodujecie, albowiem będziecie nasyceni. Błogosławieni, którzy teraz płaczecie, albowiem śmiać się będziecie. Błogosławieni jesteście, gdy ludzie was znienawidzą i gdy was wyłączą spośród siebie, gdy zelżą was i z powodu Syna Człowieczego odrzucą z pogardą wasze imię jako niecne: cieszcie się i radujcie w owym dniu, bo wielka jest wasza nagroda w niebie. Tak samo bowiem przodkowie ich czynili prorokom. Natomiast biada wam, bogaczom, bo odebraliście już pociechę waszą. Biada wam, którzy teraz jesteście syci, albowiem głód cierpieć będziecie. Biada wam, którzy się teraz śmiejecie, albowiem smucić się i płakać będziecie Biada wam, gdy wszyscy ludzie chwalić was będą. Tak samo bowiem przodkowie ich czynili fałszywym prorokom".</p>

## Noise removal

In [32]:
import re

text = str(ewangelia)

#removing html tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

#removing proverbs
def cleanbrackets (raw):
    cleaner = re.compile('\([^)]*\)')
    cleantext = re.sub(cleaner, '',raw)
    return cleantext

In [33]:
clean_text = cleanhtml(text)
clean_text = cleanbrackets(clean_text)
print(clean_text)


		Jezus podniósł oczy na swoich uczniów i mówił: "Błogosławieni jesteście, ubodzy, albowiem do was należy królestwo Boże. Błogosławieni, którzy teraz głodujecie, albowiem będziecie nasyceni. Błogosławieni, którzy teraz płaczecie, albowiem śmiać się będziecie. Błogosławieni jesteście, gdy ludzie was znienawidzą i gdy was wyłączą spośród siebie, gdy zelżą was i z powodu Syna Człowieczego odrzucą z pogardą wasze imię jako niecne: cieszcie się i radujcie w owym dniu, bo wielka jest wasza nagroda w niebie. Tak samo bowiem przodkowie ich czynili prorokom. Natomiast biada wam, bogaczom, bo odebraliście już pociechę waszą. Biada wam, którzy teraz jesteście syci, albowiem głód cierpieć będziecie. Biada wam, którzy się teraz śmiejecie, albowiem smucić się i płakać będziecie Biada wam, gdy wszyscy ludzie chwalić was będą. Tak samo bowiem przodkowie ich czynili fałszywym prorokom".


## Tokenization & Segmentation

To do: 
- tokenize the text

## Normalization

To do:
- stemming and lemmatisation

### Lemmatyzacja (lemmatisation) 
Algorytm znajdowania lemmy, formy podstawowej wyrazu w obszarze części mowy którą reprezentuje. Inaczej: wyszukiwanie formy kanonicznej leksemu. W komputerowej analizie języka naturalnego lemmatyzacja jest operacją bardziej precyzyjną niż stemmatyzacja. 

### Stemmatyzacja (stemming) 
Sprowadzenie wyrazu reprezentującego dowolną część mowy do stemu, rdzenia postaci źródłowej, najczęściej rzeczownika, bądź niekiedy nawet sekwencji znaków nie mającej samodzielnego znaczenia w języku naturalnym. 

### Stop words

In [8]:
polish_stops = codecs.open("polishStopWords",'r','utf-8')

In [9]:
ps = polish_stops.read().split('\r\n')

In [10]:
milosc = codecs.open("milosc",'r','utf-8')

In [11]:
milujcie = codecs.open("milujcie",'r','utf-8')

In [12]:
milujcie_sent = milujcie.read().split('\r\n')

In [13]:
song = ""
for sentence in milujcie_sent:
    song = song + sentence

In [14]:
text = milosc.read()
#text = song

In [15]:
tokens = word_tokenize(text)
lower_tokens = [t.lower() for t in tokens]

### Lemmatization

In [16]:
# Retain alphabetic words: alpha_only
alpha_only = [t for t in lower_tokens if t.isalpha()]

In [17]:
# Remove all stop words: no_stops
no_stops = [t for t in alpha_only if t not in ps]

In [18]:
#bag of words
bow = Counter(no_stops)

In [19]:
bow.most_common(20)

[('czyńcie', 3),
 ('należy', 3),
 ('wdzięczność', 3),
 ('grzesznicy', 3),
 ('będziecie', 3),
 ('miłujcie', 2),
 ('waszych', 2),
 ('nieprzyjaciół', 2),
 ('zwrotu', 2),
 ('miłują', 2),
 ('jakaż', 2),
 ('czynią', 2),
 ('samo', 2),
 ('powiadam', 1),
 ('słuchacie', 1),
 ('nienawidzą', 1),
 ('błogosławcie', 1),
 ('przeklinają', 1),
 ('módlcie', 1),
 ('oczerniają', 1)]

In [20]:
milosc.close
milujcie.close

<function BufferedReader.close>

## Topic identification

To do:
- bag of words
- tf-idf
- Named-entity recognition

## Classification ?